# Prep

In [27]:
import polars as pl
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.multioutput import MultiOutputClassifier
from sklearn.metrics import classification_report, confusion_matrix, multilabel_confusion_matrix, precision_recall_fscore_support, accuracy_score
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.model_selection import KFold, cross_val_score


In [2]:
df_train_raw = pl.read_csv("/home/zacharymarkofsky/Documents/GitHub/SIOP_ML_2023_Discord/data/cleaned_smushed_words.csv",null_values="NA")
# df_train_raw = pl.read_csv("/home/zacharymarkofsky/Documents/GitHub/SIOP_ML_2023_Discord/data/train_pub.csv",null_values="NA")

In [3]:
df_train_raw.head(5)

response_id,rating_chooses_appropriate_action,rating_commits_to_action,rating_gathers_information,rating_identifies_issues_opportunities,rating_interprets_information,rating_involves_others,rating_decision_making_final_score,text_exercise_4,text_exercise_5,text_exercise_6,text_exercise_7,text_exercise_8,text_exercise_9,text_exercise_10,text_exercise_11,text_exercise_12,text_exercise_13,text_exercise_14,text_exercise_15,text_exercise_16,text_exercise_17,text_exercise_18,text_exercise_19,text_exercise_final
str,i64,i64,i64,i64,i64,i64,i64,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str
"""NlmLRJzhnNt""",2,2,2,2,2,3,2,null,"""hi Tracy this …",null,null,"""hi Taylor that…","""Taylor Kim i s…","""Haley i like b…",null,null,"""Kim Taylor at …",null,null,"""Taylor please …",null,"""Taylor please …","""Kim please mee…","""see answers or…"
"""mQoZghwfMUT""",4,3,4,3,4,2,6,null,"""hi Tracy i hap…",null,null,"""hi tallyho no …",null,"""hi rm Roth i c…",null,null,null,null,null,"""hi Taylor stop…",null,"""hi Taylor it p…","""ling stop firs…","""original messa…"
"""FjIOCrTtDgF""",2,2,2,2,2,2,2,null,null,"""j please can w…","""j we look toge…",null,"""i look matter …",null,"""great i think …","""thank i see so…",null,"""thank keeping …",null,null,"""thank you ill …",null,"""wait let go ta…","""no focus team …"
"""CQGLdZnCHcZ""",3,3,3,2,2,2,3,null,null,"""hi Caxton let …","""hi Patton get …",null,"""hi Norman susp…",null,"""hi foster let …","""Jennifer infor…",null,"""hi Caxton can …","""hi Isaac let s…",null,"""hi Bern i thin…",null,"""h dais ha righ…",null
"""LERqIHLgqYm""",3,3,2,3,2,2,3,null,null,null,"""j hr along fin…",null,"""Alex i appreci…",null,"""Shea based rec…","""Jennifer thank…",null,"""j thank effort…","""Isaac by insta…",null,null,null,"""Vic i suggest …","""original messa…"


In [4]:
df_train_null  = df_train_raw.filter((pl.col('rating_chooses_appropriate_action').is_null()) |(pl.col('rating_identifies_issues_opportunities').is_null()) |(pl.col('rating_involves_others').is_null()))

In [5]:
df_train_null.describe()

describe,response_id,rating_chooses_appropriate_action,rating_commits_to_action,rating_gathers_information,rating_identifies_issues_opportunities,rating_interprets_information,rating_involves_others,rating_decision_making_final_score,text_exercise_4,text_exercise_5,text_exercise_6,text_exercise_7,text_exercise_8,text_exercise_9,text_exercise_10,text_exercise_11,text_exercise_12,text_exercise_13,text_exercise_14,text_exercise_15,text_exercise_16,text_exercise_17,text_exercise_18,text_exercise_19,text_exercise_final
str,str,f64,f64,f64,f64,f64,f64,f64,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str
"""count""","""84""",84.0,84.0,84.0,84.0,84.0,84.0,84.0,"""84""","""84""","""84""","""84""","""84""","""84""","""84""","""84""","""84""","""84""","""84""","""84""","""84""","""84""","""84""","""84""","""84"""
"""null_count""","""0""",79.0,0.0,0.0,79.0,0.0,84.0,0.0,"""65""","""64""","""28""","""32""","""66""","""23""","""65""","""31""","""30""","""66""","""26""","""32""","""66""","""41""","""64""","""10""","""8"""
"""mean""",null,2.8,2.964286,2.630952,2.8,2.321429,null,3.047619,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null
"""std""",null,0.83666,0.501718,0.596774,0.447214,0.541322,null,1.051731,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null
"""min""","""ARUBJmGmmGJ""",2.0,2.0,2.0,2.0,2.0,null,2.0,"""Karen earlier …","""Tracy great jo…","""Cory understan…","""Jamie we pick …","""Taylor excelle…","""Alex font worr…","""Haley based in…","""OK sound good …","""Jennifer after…","""Haley after re…","""Jamie good mor…","""Harley can ple…","""Taylor i sugge…","""Cory after rea…","""Taylor i can s…","""Haley it looks…","""1 as i receive…"
"""max""","""zxlxYLueJlV""",4.0,4.0,4.0,3.0,4.0,null,6.0,"""team i like se…","""thank much inf…","""we work someth…","""we see employe…","""wow it sounds …","""thanks Alex i …","""please read em…","""thank Shea res…","""unfortunately …","""sandy three wo…","""your informati…","""this go""","""this disrespec…","""thanks i commu…","""when tom weede…","""wow terrible n…","""these challeng…"
"""median""",null,3.0,3.0,3.0,3.0,2.0,null,3.0,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null


In [6]:
df_train_prepped = df_train_raw.filter(~pl.col('rating_chooses_appropriate_action').is_null()).filter(~pl.col('rating_identifies_issues_opportunities').is_null()).filter(~pl.col('rating_involves_others').is_null())

In [7]:
df_train_prepped.describe()

describe,response_id,rating_chooses_appropriate_action,rating_commits_to_action,rating_gathers_information,rating_identifies_issues_opportunities,rating_interprets_information,rating_involves_others,rating_decision_making_final_score,text_exercise_4,text_exercise_5,text_exercise_6,text_exercise_7,text_exercise_8,text_exercise_9,text_exercise_10,text_exercise_11,text_exercise_12,text_exercise_13,text_exercise_14,text_exercise_15,text_exercise_16,text_exercise_17,text_exercise_18,text_exercise_19,text_exercise_final
str,str,f64,f64,f64,f64,f64,f64,f64,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str
"""count""","""1322""",1322.0,1322.0,1322.0,1322.0,1322.0,1322.0,1322.0,"""1322""","""1322""","""1322""","""1322""","""1322""","""1322""","""1322""","""1322""","""1322""","""1322""","""1322""","""1322""","""1322""","""1322""","""1322""","""1322""","""1322"""
"""null_count""","""0""",0.0,0.0,0.0,0.0,0.0,0.0,0.0,"""1162""","""1116""","""384""","""408""","""1158""","""280""","""1126""","""334""","""422""","""1137""","""346""","""327""","""1150""","""526""","""1112""","""147""","""291"""
"""mean""",null,2.763238,3.098336,2.708775,2.67171,2.471256,2.804085,3.231467,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null
"""std""",null,0.609596,0.6172,0.651603,0.564883,0.623419,0.674089,1.325568,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null
"""min""","""AANbbjXjOFn""",1.0,1.0,1.0,1.0,2.0,2.0,1.0,"""AL per previou…","""Kim Taylor ple…","""Caxton as lead…","""0jj 0 at point…","""Jerry winters …","""Alex Alex i ap…","""Haley Blaine d…","""07 00 08 00 xr…","""Caxton can dis…","""Ashley Payne s…","""AL what know i…","""AL Isaac mi aw…","""Haley i situat…","""0 0 hi Jamie 0…","""Taylor actuall…","""AL Howard as p…","""0 0 original m…"
"""max""","""zyCojMdXzDA""",4.0,4.0,4.0,4.0,4.0,4.0,7.0,"""you need conve…","""very interesti…","""your immediate…","""you right thes…","""yes wonderful …","""your concern v…","""yes wonderful …","""yes Shea April…","""your bring goo…","""the three leve…","""you put lot ef…","""yes can move f…","""wait minute Ta…","""wine standard …","""while agree so…","""yes thank u u …","""yes i tried ha…"
"""median""",null,3.0,3.0,3.0,3.0,2.0,3.0,3.0,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null


In [8]:
df_train_prepped.columns

['response_id',
 'rating_chooses_appropriate_action',
 'rating_commits_to_action',
 'rating_gathers_information',
 'rating_identifies_issues_opportunities',
 'rating_interprets_information',
 'rating_involves_others',
 'rating_decision_making_final_score',
 'text_exercise_4',
 'text_exercise_5',
 'text_exercise_6',
 'text_exercise_7',
 'text_exercise_8',
 'text_exercise_9',
 'text_exercise_10',
 'text_exercise_11',
 'text_exercise_12',
 'text_exercise_13',
 'text_exercise_14',
 'text_exercise_15',
 'text_exercise_16',
 'text_exercise_17',
 'text_exercise_18',
 'text_exercise_19',
 'text_exercise_final']

In [9]:

df_train_prepped = df_train_prepped.with_columns(pl.concat_str([
    pl.col('text_exercise_4').fill_null(pl.lit('')),
    pl.col('text_exercise_5').fill_null(pl.lit('')),
    pl.col('text_exercise_6').fill_null(pl.lit('')),
    pl.col('text_exercise_7').fill_null(pl.lit('')),
    pl.col('text_exercise_8').fill_null(pl.lit('')),
    pl.col('text_exercise_9').fill_null(pl.lit('')),
    pl.col('text_exercise_10').fill_null(pl.lit('')),
    pl.col('text_exercise_11').fill_null(pl.lit('')),
    pl.col('text_exercise_12').fill_null(pl.lit('')),
    pl.col('text_exercise_13').fill_null(pl.lit('')),
    pl.col('text_exercise_14').fill_null(pl.lit('')),
    pl.col('text_exercise_15').fill_null(pl.lit('')),
    pl.col('text_exercise_16').fill_null(pl.lit('')),
    pl.col('text_exercise_17').fill_null(pl.lit('')),
    pl.col('text_exercise_18').fill_null(pl.lit('')),
    pl.col('text_exercise_19').fill_null(pl.lit('')),
    pl.col('text_exercise_final').fill_null(pl.lit('')),
    ],
    separator=" ").alias('all_answers'))

In [10]:
df_train_prepped = df_train_prepped.select('rating_chooses_appropriate_action',
'rating_commits_to_action',
 'rating_gathers_information',
 'rating_identifies_issues_opportunities',
 'rating_interprets_information',
 'rating_involves_others',
 'all_answers').to_pandas()

# Base Classifier

### Single Output

In [36]:
# Split the data into X and y
X = df_train_prepped['all_answers']
y = df_train_prepped.drop('all_answers',axis=1)

## Preprocess the text data using CountVectorizer
vectorizer = CountVectorizer()
X = vectorizer.fit_transform(X)

# Split the data into train, test, and validation sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, shuffle = True)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.25, random_state=42)

# Train a random forest classifier on the training set
rf = RandomForestClassifier(random_state=42)
for col in y.columns:
    rf.fit(X_train, y_train[col])

    # Predict on validation set
    y_pred_val = rf.predict(X_val)

    cm = confusion_matrix(np.array(y_val[col]), y_pred_val)
    print(f"Confusion matrix for column: {col}:")
    print(cm)

    cr = classification_report(np.array(y_val[col]), y_pred_val,zero_division=1)
    print(f"\n Classification report for column: {col}:")
    print(cr)

    # Calculate the precision, recall, and F1 score for each label separately
    precision, recall, f1_score, _ = precision_recall_fscore_support(np.array(y_val[col]), y_pred_val, zero_division=1)
    print(f"Precision, recall, and F1 score for column: {col}:")
    print(f"Precision: {precision}")
    print(f"Recall: {recall}")
    print(f"F1 score: {f1_score} \n")



Confusion matrix for column: rating_chooses_appropriate_action:
[[  0   1   1   0]
 [  0  21  60   0]
 [  0  13 145   0]
 [  0   3  21   0]]

 Classification report for column: rating_chooses_appropriate_action:
              precision    recall  f1-score   support

           1       1.00      0.00      0.00         2
           2       0.55      0.26      0.35        81
           3       0.64      0.92      0.75       158
           4       1.00      0.00      0.00        24

    accuracy                           0.63       265
   macro avg       0.80      0.29      0.28       265
weighted avg       0.65      0.63      0.56       265

Precision, recall, and F1 score for column: rating_chooses_appropriate_action:
Precision: [1.         0.55263158 0.63876652 1.        ]
Recall: [0.         0.25925926 0.91772152 0.        ]
F1 score: [0.         0.35294118 0.75324675 0.        ] 

Confusion matrix for column: rating_commits_to_action:
[[  0   0   1   0]
 [  0   0  39   1]
 [  0   4 15

### Multi-Output

In [29]:
# Split the data into X and y
X = df_train_prepped['all_answers']
y = df_train_prepped.drop('all_answers',axis=1)

## Preprocess the text data using CountVectorizer
vectorizer = CountVectorizer()
X = vectorizer.fit_transform(X)

# Split the data into train, test, and validation sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, shuffle = True)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.25, random_state=42)

# Train a random forest classifier on the training set
rf = RandomForestClassifier(random_state=42)
clf = MultiOutputClassifier(rf)
clf.fit(X_train, y_train)

# Predict on validation set
y_pred_val = clf.predict(X_val)


In [44]:
for i in range(np.array(y_val).shape[1]):
    # Calculate the confusion matrix for each column separately
    
    cm = confusion_matrix(np.array(y_val)[:, i], y_pred_val[:, i])
    print(f"Confusion matrix for column {i}:")
    print(cm)

    cr = classification_report(np.array(y_val)[:, i], y_pred_val[:, i],zero_division=1)
    print(f"\n Classification report for column {i}:")
    print(cr)

    # Calculate the precision, recall, and F1 score for each label separately
    precision, recall, f1_score, _ = precision_recall_fscore_support(np.array(y_val)[:, i], y_pred_val[:, i], zero_division=1)
    print(f"Precision, recall, and F1 score for column {i}:")
    print(f"Precision: {precision}")
    print(f"Recall: {recall}")
    print(f"F1 score: {f1_score} \n")



Confusion matrix for column 0:
[[  0   1   1   0]
 [  0  21  60   0]
 [  0  13 145   0]
 [  0   3  21   0]]

 Classification report for column 0:
              precision    recall  f1-score   support

           1       1.00      0.00      0.00         2
           2       0.55      0.26      0.35        81
           3       0.64      0.92      0.75       158
           4       1.00      0.00      0.00        24

    accuracy                           0.63       265
   macro avg       0.80      0.29      0.28       265
weighted avg       0.65      0.63      0.56       265

Precision, recall, and F1 score for column 0:
Precision: [1.         0.55263158 0.63876652 1.        ]
Recall: [0.         0.25925926 0.91772152 0.        ]
F1 score: [0.         0.35294118 0.75324675 0.        ] 

Confusion matrix for column 1:
[[  0   0   1   0]
 [  0   0  39   1]
 [  0   4 151  19]
 [  0   1  29  20]]

 Classification report for column 1:
              precision    recall  f1-score   support

   

## K-Folds CV

In [67]:
# Split the data into X and y
X = df_train_prepped['all_answers']
y = df_train_prepped.drop('all_answers',axis=1)

## Preprocess the text data using CountVectorizer
vectorizer = CountVectorizer()
X = vectorizer.fit_transform(X)

# Split the data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, shuffle=True)

# Set the number of folds for cross-validation
k = 5

# Initialize lists to store precision, recall, and F1 scores for each fold and each label
precision_scores = []
recall_scores = []
f1_scores = []

# Loop over each column in y
for col in y.columns:

    # Initialize lists to store precision, recall, and F1 scores for each fold
    precision_scores_col = []
    recall_scores_col = []
    f1_scores_col = []
    cm_scores_col = []

    # Split the data into k folds
    kf = KFold(n_splits=k, shuffle=True, random_state=42)

    print(f"Confusion Matrixes for column: {col}")
    # Loop over each fold
    for fold_idx, (train_idx, val_idx) in enumerate(kf.split(X_train, y_train[col])):
        # Get the training and validation data for this fold
        X_train_fold = X_train[train_idx]
        y_train_fold = y_train[col].iloc[train_idx]
        X_val_fold = X_train[val_idx]
        y_val_fold = y_train[col].iloc[val_idx]

        # Train a random forest classifier on the training data for this fold
        rf = RandomForestClassifier(random_state=42)
        rf.fit(X_train_fold, y_train_fold)

        # Predict on validation set for this fold
        y_pred_val = rf.predict(X_val_fold)

        # Calculate precision, recall, and F1 score for this fold and this label
        precision, recall, f1_score, _ = precision_recall_fscore_support(y_val_fold, y_pred_val, zero_division=1)
        cm = confusion_matrix(y_val_fold, y_pred_val)

        # Append scores for this fold and this label to lists
        precision_scores_col.append(precision)
        recall_scores_col.append(recall)
        f1_scores_col.append(f1_score)
        cm_scores_col.append(cm)

        # Print the confusion matrix for this fold and this label
        print(f"fold: {fold_idx}")
        print(cm)

    # Calculate the average precision, recall, and F1 score across all folds for this column
    precision_avg = np.mean(np.concatenate(precision_scores_col, axis=0 ))
    recall_avg = np.mean(np.concatenate(recall_scores_col, axis=0 ))
    f1_avg = np.mean(np.concatenate(f1_scores_col, axis=0 ))

    # Append the average scores for this column to the overall lists
    precision_scores.append(precision_avg)
    recall_scores.append(recall_avg)
    f1_scores.append(f1_avg)
    
    print(f"Average precision, recall, and F1 score for column: {col}")
    print(f"Precision: {precision_avg}")
    print(f"Recall: {recall_avg}")
    print(f"F1 score: {f1_avg}\n")

Confusion Matrixes for column: rating_chooses_appropriate_action
fold: 0
[[  0   1   1   0]
 [  0  16  51   0]
 [  0  11 113   0]
 [  0   2  17   0]]
fold: 1
[[  0   1   0   0]
 [  0  17  54   0]
 [  0  11 111   0]
 [  0   0  18   0]]
fold: 2
[[  0   1   2   0]
 [  0  19  46   0]
 [  0   9 117   0]
 [  0   2  15   0]]
fold: 3
[[  0   1   1   0]
 [  0  11  57   1]
 [  0  12 102   0]
 [  0   0  26   0]]
fold: 4
[[ 13  54   0]
 [ 14 117   0]
 [  0  13   0]]
Average precision, recall, and F1 score for column: rating_chooses_appropriate_action
Precision: 0.7228395489698005
Recall: 0.2979770975052669
F1 score: 0.27541468916778916

Confusion Matrixes for column: rating_commits_to_action
fold: 0
[[  0   0   1   0]
 [  0   0  29   1]
 [  0   3 128   7]
 [  0   0  29  14]]
fold: 1
[[  0  36   1]
 [  0 117   7]
 [  0  34  17]]
fold: 2
[[  0  21   0]
 [  0 121   8]
 [  0  47  14]]
fold: 3
[[  0   0   1   0]
 [  0   0  29   0]
 [  0   0 125   5]
 [  0   0  35  16]]
fold: 4
[[  0  30   1]
 [  0 124 